In [7]:
args = {
    "sourcebucket": "cfstack-projeto-integrador-grupo-a-raw",
    "sourceprefix": "tlc_trip/yellow_tripdata",
    "sourcedatabase": "tlc_trip",
    "sourcezonetable": "taxi_zone_lookup",
    "targetbucket": "cfstack-projeto-integrador-grupo-a-trusted",
    "targetdatabase": "trusted_tlc_trip",
    "targettable": "yellow_tripdata",
    "month": "2011-03"
}

Calculation started (calculation_id=64caf170-3269-3fbf-e7f1-935f81f26d4c) in (session=b2caf16a-46ea-1bbd-43c2-c762619686e2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
glue_script(args)

Calculation started (calculation_id=b6caf170-3827-c192-dfcd-06c95810394e) in (session=b2caf16a-46ea-1bbd-43c2-c762619686e2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

In [1]:
def glue_script(args):
    
    source_bucket     = args['sourcebucket']
    source_prefix     = args['sourceprefix']
    source_database   = args['sourcedatabase']
    source_zone_table = args['sourcezonetable']
    target_bucket     = args['targetbucket']
    target_database   = args['targetdatabase']
    target_table      = args['targettable']
    month             = args['month']

    spark.read.parquet(f's3://{source_bucket}/{source_prefix}/{month}.parquet').createOrReplaceTempView('temp_yellow_tripdata')
    
    df = spark.sql(
    f'''
        SELECT
            CAST(yt.VendorID AS INT) AS vendorid,
            CAST(yt.tpep_pickup_datetime AS TIMESTAMP) AS pickup_datetime,
            CAST(yt.tpep_dropoff_datetime AS TIMESTAMP) AS dropoff_datetime,
            CAST(yt.passenger_count AS INT) AS passenger_count,
            CAST(yt.trip_distance AS DOUBLE) AS trip_distance,
            CAST(yt.RatecodeID AS INT) AS rate_code_id,
            CAST(yt.store_and_fwd_flag AS STRING) AS store_and_fwd_flag,
            CAST(puz.borough AS STRING) AS pickup_neighborhood,
            CAST(puz.zone AS STRING) AS pickup_zone,
            CAST(puz.service_zone AS STRING) AS pickup_service_zone,
            CAST(doz.borough AS STRING) AS dropoff_neighborhood,
            CAST(doz.zone AS STRING) AS dropoff_zone,
            CAST(doz.service_zone AS STRING) AS dropoff_service_zone,
            CAST(yt.payment_type AS INT) AS payment_type,
            CAST(yt.fare_amount AS DOUBLE) AS fare_amount,
            CAST(yt.extra AS DOUBLE) AS extra,
            CAST(yt.mta_tax AS DOUBLE) AS mta_tax,
            CAST(yt.tip_amount AS DOUBLE) AS tip_amount,
            CAST(yt.tolls_amount AS DOUBLE) AS tolls_amount,
            CAST(yt.improvement_surcharge AS DOUBLE) AS improvement_surcharge,
            CAST(yt.total_amount AS DOUBLE) AS total_amount,
            CAST(yt.congestion_surcharge AS DOUBLE) AS congestion_surcharge,
            CAST(yt.airport_fee AS DOUBLE) AS airport_fee,
            '{month}' AS partition_dt
        FROM
            temp_yellow_tripdata yt
            INNER JOIN {source_database}.{source_zone_table} puz ON CAST(yt.PULocationID AS INT) = puz.locationid
            INNER JOIN {source_database}.{source_zone_table} doz ON CAST(yt.DOLocationID AS INT) = doz.locationid
    '''
    ).cache()
    
    df.write. \
        mode('append'). \
        format('parquet'). \
        option('compression', 'snappy'). \
        option('path', f's3://{target_bucket}/{target_database}/{target_table}/'). \
        partitionBy('partition_dt'). \
        saveAsTable(f'{target_database}.{target_table}')

Calculation started (calculation_id=e0caf16b-cc28-1570-f965-3d0ddf349df0) in (session=b2caf16a-46ea-1bbd-43c2-c762619686e2). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
